<a href="https://colab.research.google.com/github/AntonioHenriqueAC/Barcode_Number-Recognition/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baixando as bibliotecas necessárias

In [0]:
!pip3 install tensorflow-gpu==1.13.1

In [0]:
!pip3 install imageai --upgrade

In [0]:
!wget https://github.com/AntonioHenriqueAC/tcc/releases/download/1.0/Colab.zip
!unzip "Colab.zip"
!rm -rf Colab.zip

# Indetificação dos Barcodes


## 1. Criando as pastas ##

In [0]:
import os

path, dirs, files = next(os.walk("Colab/Corridas"))
file_count_corridas = len(dirs)
if(file_count_corridas == 0):
  print("Número de corridas na pasta: ", file_count_corridas)
  file_count_corridas = 1
else:
  print("Número de corridas na pasta: ", file_count_corridas)
  file_count_corridas = len(dirs) + 1

newFolderCorrida = str(file_count_corridas)
folderCorrida = "Colab/Corridas/Corrida_" + newFolderCorrida
print("..")
print("..")
print("..")
!mkdir $folderCorrida
print("Criando pasta 'Corrida_"+newFolderCorrida+"'")

folderTagsJson = "Colab/Corridas/Corrida_" + newFolderCorrida + "/tags_json"
print("..")
print("..")
print("..")
try:
  !mkdir $folderTagsJson
  print("Criando pasta '/tags_json'")
except:
  print("Já existe a pasta '/tags_json'")

folderTagsDetected = "Colab/Corridas/Corrida_" + newFolderCorrida + "/tags_detected"
print("..")
print("..")
print("..")
try:
  !mkdir $folderTagsDetected
  print("Criando pasta '/tags_detected'")
except:
  print("Já existe a pasta '/tags_detected'")

print("..")
print("..")
print("..")
try:
  !mkdir barcode-detected-objects
  print("Criando pasta '/barcode-detected-objects'")
except:
  print("Já existe a pasta '/barcode-detected-objects'")

## 2. Setando as variaveis ##


In [0]:
modelPath = "Colab/models/barcode/detection_model-ex-013--loss-0001.995.h5"
JsonPath = "Colab/json/detection_config_barcode.json"

Image_Number = newFolderCorrida
InputImage = "Colab/Imagens_Originais/img"+ Image_Number +".jpg"

## 3. Função de interseção sobre união e Função de eliminar quadrados sobrepostos.

In [0]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

def boxArea(boxA, boxB):
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	return boxAArea/boxBArea

def removeDuplicatesLoop(detections):
	i=0
	k=0
	for i, detection in enumerate(detections):
		for k, detection in enumerate(detections):
					inter = bb_intersection_over_union(detections[i]["box_points"] , detections[k]["box_points"])
					if(inter > float(0.01) and inter < float(1)):
						if(boxArea(detections[i]["box_points"] , detections[k]["box_points"]) > 1):
							detections.pop(k)
						else:
							detections.pop(i)
							removeDuplicatesLoop(detections)

## 4. Treinando modelo e recortando os codigos de barras da imagem original ##


In [0]:
from imageai.Detection.Custom import CustomObjectDetection
from prettytable import PrettyTable

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(modelPath)
detector.setJsonPath(JsonPath)
detector.loadModel()

detections = detector.detectObjectsFromImage(input_image = InputImage, 
                                                           output_image_path="barcode-detected-OriginalBoxes.jpg",
                                                           minimum_percentage_probability=30)
for e in detections:
  e['box_points'][0] = e['box_points'][0] - 50 ## x1
  e['box_points'][1] = e['box_points'][1] - 50 ## y1
  e['box_points'][2] = e['box_points'][2] + 50 ## x2
  e['box_points'][3] = e['box_points'][3] + 50 ## y2

removeDuplicatesLoop(detections)

header = PrettyTable(["Nome", "Porcentagem ", "box_points"])
for detection in detections:
  header.add_row([detection["name"], detection["percentage_probability"], detection["box_points"]])

img = PrettyTable(["                 Barcodes identificados                 "])
img.add_row([len(detections)])

print("")
print("")
print(header)
print("")
print(img)
print("")

## 5. Identificando as novas Bounding Box, recortando da imagem original e exportando os arquivos para pasta "/tags_images" 

In [0]:
%matplotlib inline
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import skimage.segmentation
import cv2

Image_with_boxes = "barcode-detected-OriginalBoxes.jpg"

img = skimage.io.imread(InputImage)
bbox = []

for e in detections:
    bbox.append(e['box_points'])

for i, (x1,y1,x2,y2) in enumerate(bbox):

    if i in {0,1,2,3,4,5,6,7,8,9}:
      i = '00' + str(i)
    
    if(int(i) > 9 and int(i) < 100):
      i = '0' + str(i)

    img2 = cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 0)
    out = img[y1:y2, x1:x2]

    cv2.imwrite('barcode-detected-objects/barcode-00'+str(i)+'.jpg', out)

cv2.imwrite('barcode-detected-New.jpg', img)

print("Boxes recortados com sucesso!")
print("..")
print("..")
print("..")
print("Imagem = 'barcode-detected-New.jpg' gerada.")

## 6. Movendo as pastas para o diretório correto. ##


In [0]:
!mv /content/barcode-detected-OriginalBoxes.jpg $folderCorrida
!mv /content/barcode-detected-New.jpg $folderCorrida
!mv /content/barcode-detected-objects $folderCorrida

barcode = "Colab/Corridas/Corrida_" + newFolderCorrida + "/barcode-detected-objects"
images = "Colab/Corridas/Corrida_" + newFolderCorrida + "/tags_images"
!mv  $barcode $images

print("Pastas movidas para o drive com sucesso!")

# Identificação dos Números


## 1. Contar numero de arquivos nas pastas

In [0]:
import os
from prettytable import PrettyTable

path, dirs, files = next(os.walk("Colab/Corridas"))
pasta = PrettyTable(["Corridas na pasta", "Status"])
for i in range(0, len(dirs)):
  pathJSON, dirsJSON, filesJSON = next(os.walk("Colab/Corridas"))
  if(len(filesJSON)>0):
    status = "Treinada"
  else:
    status = "Inadimplente"
  pasta.add_row([dirs[i], status])

print("")
print(pasta)
print("")
print("")
print("Qual corrida deseja identificar os codigos de barras? (Informe o número da pasta)")
corrida = input()
print("")

path, dirs, files = next(os.walk("Colab/Corridas/Corrida_" + str(corrida) +"/tags_images"))
file_count_tag = len(files)
tags = PrettyTable(["Treinar", "Tags"])
tags.add_row(["Corrida_"+str(corrida), str(file_count_tag)])

print("")
print(tags)
print("")

## 2. Função para fazer a diferença entre distancia dos números e deletar o de menor Acurácia

In [0]:
def removeDuplicates():
    i=0
    for detection in detections:
      if( i != len(detections)-1 ):
          
          Xa = (detections[i]['box_points'][0] + detections[i]['box_points'][2])/2
          Ya = (detections[i]['box_points'][1] + detections[i]['box_points'][3])/2
          Xb = (detections[i+1]['box_points'][0] + detections[i+1]['box_points'][2])/2
          Yb = (detections[i+1]['box_points'][1] + detections[i+1]['box_points'][3])/2

          Dab = ( (Xb-Xa)**2 + (Yb - Ya)**2 )**1/2
          if(Dab < float(10)):
              if(detections[i]["percentage_probability"] < detections[i+1]["percentage_probability"]):
                  detections.pop(i)
              else:
                  detections.pop(i+1)
      i=i+1

## 3. Identificando os numeros.

In [0]:
from imageai.Detection.Custom import CustomObjectDetection
import json
from prettytable import PrettyTable

modelPath = "Colab/models/number/detection_model-ex-034--loss-0012.508.h5"
JsonPath = "Colab/json/detection_config_number.json"

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(modelPath)
detector.setJsonPath(JsonPath)
detector.loadModel()

for x in range(file_count_tag):

  i=0
  soma=0
  numero=''
  execucoes = 0

  if x in {0,1,2,3,4,5,6,7,8,9}:
    x = '00' + str(x)
  
  if(int(x) > 9 and int(x) < 100):
    x = '0' + str(x)

  detections = detector.detectObjectsFromImage(input_image="Colab/Corridas/Corrida_"+str(corrida)+"/tags_images/barcode-00"+str(x)+".jpg", 
                                          output_image_path="Colab/Corridas/Corrida_"+str(corrida)+"/tags_detected/digit-detected-"+str(x)+".jpg", 
                                          display_percentage_probability=False, 
                                          minimum_percentage_probability=60)
  def boxPoints(e):
    return e['box_points']

  detections.sort(key=boxPoints)

  menor = 100
  while(len(detections) > 13):
    execucoes = execucoes + 1
    if(execucoes>10000):
      for i, detection in enumerate(detections):
        if(menor > detections[i]["percentage_probability"]):
          menor = detections[i]["percentage_probability"]   
      for k, detection in enumerate(detections):
        if(detections[k]["percentage_probability"] == menor):
          detections.pop(k)
      break
    removeDuplicates()

  img = PrettyTable(["                     Imagem                    "])
  img.add_row([x])
  header = PrettyTable(["Número", "Porcentagem ", "box_points"])

  for i in range(len(detections)):
    if(detections[i]["percentage_probability"] < 99):
      detections[i].update({"alert" : "true"})
    else:
      detections[i].update({"alert" : "false"})
    header.add_row([detections[i]["name"], detections[i]["percentage_probability"], detections[i]["box_points"]])
    soma = soma + detections[i]["percentage_probability"]
    numero = numero + ''.join(detections[i]["name"])

  if(len(detections) != 0):
    media = soma/len(detections)
  table = PrettyTable(["Números", "Quantidade", "Média"])
  table.add_row([numero, len(detections), media])

  print("")
  print("")
  print(img)
  print("")
  print(header)
  print("")
  print(table)
  print("")
  print("")
  print("============================================================")

  media = {"media": media}
  imagem = {"imagem": x}
  numero = {"numero": numero}
  detections.append(media)
  detections.append(imagem)
  detections.append(numero)

  with open('Colab/Corridas/Corrida_'+str(corrida)+'/tags_json/tag-'+ str(x) +'.json', 'w', encoding='utf-8') as f:
    json.dump(detections, f, ensure_ascii=False, indent=4)

# Validação


In [0]:
print("Corrida "+str(corrida)+" identificada com sucesso!!!")